In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD") 
dbutils.widgets.text("target_table", "f_revenue_pos_b2b")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDSDBL02"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b
(
  hash_id BIGINT,
  calday DATE,
  store_id STRING,
  bill_type STRING,
  customer_master_id STRING,
  distr_chan STRING,
  product_id STRING,
  m_tax_amt FLOAT,
  m_rev_amt FLOAT,
  m_cogs FLOAT,
  base_q FLOAT,
  subtot_1s FLOAT,
  subtot_2s FLOAT,
  subtot_3s FLOAT,
  subtot_4s FLOAT,
  subtot_5s FLOAT,
  m_vinid_earn FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_f_revenue_pos_b2b AS
SELECT 
    farm_fingerprint(
        CONCAT(
            IFNULL(CAST(calday AS STRING), ""), 
            IFNULL(CAST(store_id AS STRING), ""), 
            IFNULL(CAST(product_id AS STRING), ""), 
            IFNULL(CAST(customer_master_id AS STRING), ""), 
            IFNULL(CAST(bill_type AS STRING), ""), 
            IFNULL(CAST(distr_chan AS STRING), "")
        )
    ) hash_id,
    calday,
    store_id,
    bill_type,
    customer_master_id,
    distr_chan,
    product_id,
    SUM(tax_amount) m_tax_amt,
    SUM(netval_inv) m_rev_amt,
    SUM(cost) m_cogs,
    SUM(quant_b) base_q,
    SUM(subtot_1s) subtot_1s,
    SUM(subtot_2s) subtot_2s,
    SUM(subtot_3s) subtot_3s,
    SUM(subtot_4s) subtot_4s,
    SUM(subtot_5s) subtot_5s,
    SUM(subtot_6s) m_vinid_earn 
FROM (
    SELECT 
        calday,
        plant store_id,
        bill_type,
        sold_to customer_master_id,
        distr_chan,
        material product_id,
        tax_amount,
        netval_inv,
        cost,
        CASE 
            WHEN quant_b < 0.000001 AND quant_b > 0 THEN 0
            WHEN quant_b > -0.000001 AND quant_b < 0 THEN 0
            ELSE quant_b
        END quant_b,
        CASE 
            WHEN subtot_1s IS NULL THEN 0
            ELSE subtot_1s
        END subtot_1s,
        CASE 
            WHEN subtot_2s IS NULL THEN 0
            ELSE subtot_2s
        END subtot_2s,
        CASE 
            WHEN subtot_3s IS NULL THEN 0
            ELSE subtot_3s
        END subtot_3s,
        CASE 
            WHEN subtot_4s IS NULL THEN 0
            ELSE subtot_4s
        END subtot_4s,
        CASE 
            WHEN subtot_5s IS NULL THEN 0
            ELSE subtot_5s
        END subtot_5s,
        CASE 
            WHEN subtot_6s IS NULL THEN 0
            ELSE subtot_6s
        END subtot_6s
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdsdbl02
    WHERE calday in ({CALDAY_IN_STR})
) a 
GROUP BY calday, store_id, product_id, customer_master_id, bill_type, distr_chan
""")

In [0]:
if CALDAY_IN_STR != '':
    # Get the calday list string from temp table
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_f_revenue_pos_b2b
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.") 
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_revenue_pos_b2b can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_f_revenue_pos_b2b")